In [5]:
import os
from openai import OpenAI


In [6]:
# !pip install openai

In [7]:
OPENAI_API_KEY =''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [14]:
client = OpenAI()
client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-umBGkFDCspopHle4QFJT6aDb', bytes=14712, created_at=1711098569, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [19]:
client.fine_tuning.jobs.create(
  training_file="file-umBGkFDCspopHle4QFJT6aDb", 
  model="davinci-002"
)

FineTuningJob(id='ftjob-SAmMXwe7iIbZxEP8IKuoVvkE', created_at=1711099016, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-O3soZSo9NkqohwecviHDDLCg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-umBGkFDCspopHle4QFJT6aDb', validation_file=None, user_provided_suffix=None)

In [30]:
client.fine_tuning.jobs.retrieve("ftjob-SAmMXwe7iIbZxEP8IKuoVvkE")

FineTuningJob(id='ftjob-SAmMXwe7iIbZxEP8IKuoVvkE', created_at=1711099016, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:davinci-002:axiom-law::95VcKL4T', finished_at=1711100204, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='davinci-002', object='fine_tuning.job', organization_id='org-O3soZSo9NkqohwecviHDDLCg', result_files=['file-O0km82Mn1O9ANeXd6XfLkcCf'], status='succeeded', trained_tokens=8394, training_file='file-umBGkFDCspopHle4QFJT6aDb', validation_file=None, user_provided_suffix=None)

In [29]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-SAmMXwe7iIbZxEP8IKuoVvkE", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-ybI0jHIEXpBDeS2nwppfCuLc', created_at=1711099039, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'), FineTuningJobEvent(id='ftevent-nHFu9TNMyKTXGqE94dAce6Fk', created_at=1711099038, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-t3Zalpt5TVU334Ya7Bl8DJEd', created_at=1711099016, level='info', message='Validating training file: file-umBGkFDCspopHle4QFJT6aDb', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-OfLt76KfaLfEoBFo82iDfMOD', created_at=1711099016, level='info', message='Created fine-tuning job: ftjob-SAmMXwe7iIbZxEP8IKuoVvkE', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [53]:
!curl https://api.openai.com/v1/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{"model": "ft:davinci-002:axiom-law::95VcKL4T", "prompt": "What is the Operation and Maintenance Manual SMBU6981 about?", "temperature": 0}'

{
  "id": "cmpl-95XQkuBLqUE43DKli6FfO5zMJO1gq",
  "object": "text_completion",
  "created": 1711107174,
  "model": "ft:davinci-002:axiom-law::95VcKL4T",
  "choices": [
    {
      "text": "It is about the Operation and Maintenance Manual for the 3208TA Marine Engine",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 16,
    "total_tokens": 29
  }
}


In [1]:
!pip install -qU openai datasets transformers evaluate rouge_score


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset

ds = load_dataset("nomic-ai/gpt4all-j-prompt-generations",split="train")

Generating train split:   0%|          | 0/808812 [00:00<?, ? examples/s]

In [3]:
ds_sample = ds.select(range(1000))
ds_sample

Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 1000
})

In [7]:
import random
def format_data(data):

  formatted_data = []
  for message in data:
    msg = {"messages" :
          [
          {"role":"system",
            "content":"""You are the Yahoo platform user's assistant. Please reply users' answer using polite,clear and respectful language."""
          },
          {"role":"user",
          "content":message["prompt"]
          },
          {"role":"assistant",
          "content":message["response"]
          }
          ]
          }
    formatted_data.append(msg)
  random.shuffle(formatted_data)
  return formatted_data

formatted_data = format_data(ds_sample)
print(formatted_data[0])

{'messages': [{'role': 'system', 'content': "You are the Yahoo platform user's assistant. Please reply users' answer using polite,clear and respectful language."}, {'role': 'user', 'content': 'Produce an article summary of the following news article: Surf pool, United Arab Emirates\n\nThe Box, Margaret River Valley, Australia\n\nLance\'s Right, Mentawais Islands, Indonesia\n\nP-Pass, Federated States of Micronesia\n\nSupertubes, Jeffrey\'s Bay, South Africa\n\nPipeline, Oahu, Hawaii, United States\n\n(CNN) -- Maybe you surf, maybe you don\'t.\n\nEveryone who\'s been to the beach in the last 50 years is influenced by surf culture.\n\nIf you\'ve ever listened to a Beach Boys record (most of the guys in the group didn\'t surf) or donned a pair of baggy shorts or looked wistfully out at the breakers and dreamed of an endless summer of waves and bonfires with the perfect surfer guy or girl on your arm, you can claim a connection to surf culture.\n\nThat\'s why even if you\'re the type who s

In [17]:
type(formatted_data)

list

In [16]:
print(formatted_data[3])

{'messages': [{'role': 'system', 'content': "You are the Yahoo platform user's assistant. Please reply users' answer using polite,clear and respectful language."}, {'role': 'user', 'content': 'Produce an article summary of the following news article: Three Crystal Transport bus drivers failed drug tests but kept transporting thousands of passengers around Boston every day, according to federal investigators. Some new hires did not undergo proper drug screening; others falsified duty records to hide the fact that they exceeded the maximum hours allowed behind the wheel.\n\nThe violations piled up for five years before federal regulators in February conducted a full safety review of the Boston company — and quickly pulled its buses off the road. Crystal has since been reauthorized to operate here.\n\nCrystal Transport Inc. is just one example of the many bus companies with persistent safety violations that have been allowed to continue operating, a Globe analysis of federal data found.\n

In [13]:
import pandas as pd

df = pd.read_csv('test.csv', sep=';')

In [14]:
df.head()

,prompt,completion
0,What ROPS/FOPS Certification Means?,It provides information on ROPS/FOPS certifica...
1,What is the content of Special Publication PE...,It covers the product data sheet for Caterpill...
2,What is the focus of Special Publication SEHS6...,"It provides guidelines for the inspection, mai..."
3,What does Special Publication PEHP7506 cover?,It includes product data sheet and installatio...
4,What is the purpose of Special Publication PEH...,It provides safety manuals for Caterpillar Gea...


In [19]:
import pandas as pd
import json

json_data = []
for _, row in df.iterrows():
    message = [
        {'role': 'system', 'content': "You are the user's assistant. Please reply user`s questions"},
        {'role': 'user', 'content': row['prompt']},
        {'role': 'assistant', 'content': row['completion']}
    ]
    json_data.append({'messages': message})

json_string = json.dumps(json_data, indent=4)


In [20]:
json_file_path = "output.json"

with open(json_file_path, "w") as json_file:
    json_file.write(json_string)